In [1]:
!pip install tensorflow==2.7.0 tensorflow-gpu==2.7.0 opencv-python mediapipe sklearn matplotlib

You should consider upgrading via the 'c:\users\asus\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [7]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [8]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [9]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [10]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [11]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# Actions that we try to detect
actions = np.array(['cerewet', 'setia', 'kurus', 'baik hati', 'pintar', 'jelek', 'mewah', 'lucu', 'curiga', 'kasar', 'sombong', 'tua'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30

In [12]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [30]:
cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [27]:
cap.release()
cv2.destroyAllWindows()

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
label_map

{'cerewet': 0,
 'setia': 1,
 'kurus': 2,
 'baik hati': 3,
 'pintar': 4,
 'jelek': 5,
 'mewah': 6,
 'lucu': 7,
 'curiga': 8,
 'kasar': 9,
 'sombong': 10,
 'tua': 11}

In [16]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [17]:
np.array(sequences).shape

(720, 30, 1662)

In [18]:
np.array(labels).shape

(720,)

In [19]:
X = np.array(sequences)

In [20]:
y = to_categorical(labels).astype(int)

In [21]:
y


array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [23]:
X.shape

(720, 30, 1662)

In [24]:
y_test.shape

(36, 12)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [26]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [27]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='sigmoid', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='sigmoid'))
model.add(LSTM(64, return_sequences=False, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [28]:
actions.shape[0]

12

In [29]:
res = [.7, 0.2, 0.1]

In [30]:
actions[np.argmax(res)]

'cerewet'

In [31]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [32]:
model.fit(X_train, y_train, epochs=2500, callbacks=[tb_callback])

Epoch 1/2500
22/22 [==============================] - 13s 209ms/step - loss: 2.5453 - categorical_accuracy: 0.0848
Epoch 2/2500
22/22 [==============================] - 5s 209ms/step - loss: 2.4946 - categorical_accuracy: 0.0863
Epoch 3/2500
22/22 [==============================] - 5s 209ms/step - loss: 2.4866 - categorical_accuracy: 0.0804
Epoch 4/2500
22/22 [==============================] - 5s 217ms/step - loss: 2.4884 - categorical_accuracy: 0.0789
Epoch 5/2500
22/22 [==============================] - 5s 208ms/step - loss: 2.4876 - categorical_accuracy: 0.0775
Epoch 6/2500
22/22 [==============================] - 5s 211ms/step - loss: 2.4890 - categorical_accuracy: 0.0614
Epoch 7/2500
22/22 [==============================] - 6s 293ms/step - loss: 2.4880 - categorical_accuracy: 0.0863
Epoch 8/2500
22/22 [==============================] - ETA: 0s - loss: 2.4843 - categorical_accuracy: 0.08 - 5s 235ms/step - loss: 2.4843 - categorical_accuracy: 0.0833
Epoch 9/2500
22/22 [=============

22/22 [==============================] - 6s 274ms/step - loss: 1.9312 - categorical_accuracy: 0.1550
Epoch 140/2500
22/22 [==============================] - 5s 238ms/step - loss: 1.9307 - categorical_accuracy: 0.1681
Epoch 141/2500
22/22 [==============================] - 5s 212ms/step - loss: 1.9311 - categorical_accuracy: 0.1520
Epoch 142/2500
22/22 [==============================] - 5s 221ms/step - loss: 1.9311 - categorical_accuracy: 0.15060s - loss: 1.9252 - categorical_accuracy
Epoch 143/2500
22/22 [==============================] - 5s 233ms/step - loss: 1.9311 - categorical_accuracy: 0.1520
Epoch 144/2500
22/22 [==============================] - 5s 226ms/step - loss: 1.9294 - categorical_accuracy: 0.1740
Epoch 145/2500
22/22 [==============================] - 5s 222ms/step - loss: 1.9303 - categorical_accuracy: 0.1550
Epoch 146/2500
22/22 [==============================] - 5s 218ms/step - loss: 1.9293 - categorical_accuracy: 0.1725
Epoch 147/2500
22/22 [=========================

Epoch 209/2500
22/22 [==============================] - 5s 230ms/step - loss: 1.2800 - categorical_accuracy: 0.3728
Epoch 210/2500
22/22 [==============================] - 5s 236ms/step - loss: 1.3402 - categorical_accuracy: 0.3596
Epoch 211/2500
22/22 [==============================] - 5s 240ms/step - loss: 1.2631 - categorical_accuracy: 0.4006
Epoch 212/2500
22/22 [==============================] - 5s 228ms/step - loss: 1.2057 - categorical_accuracy: 0.4108
Epoch 213/2500
22/22 [==============================] - 5s 234ms/step - loss: 1.2070 - categorical_accuracy: 0.4108
Epoch 214/2500
22/22 [==============================] - 5s 236ms/step - loss: 1.1720 - categorical_accuracy: 0.4415
Epoch 215/2500
22/22 [==============================] - 5s 227ms/step - loss: 1.1597 - categorical_accuracy: 0.4737
Epoch 216/2500
22/22 [==============================] - 5s 221ms/step - loss: 1.1455 - categorical_accuracy: 0.4708
Epoch 217/2500
22/22 [==============================] - 5s 226ms/step - 

22/22 [==============================] - 5s 236ms/step - loss: 0.6561 - categorical_accuracy: 0.7120
Epoch 348/2500
22/22 [==============================] - 5s 233ms/step - loss: 0.6938 - categorical_accuracy: 0.6886
Epoch 349/2500
22/22 [==============================] - 5s 231ms/step - loss: 0.6625 - categorical_accuracy: 0.6901
Epoch 350/2500
22/22 [==============================] - 4s 201ms/step - loss: 0.6730 - categorical_accuracy: 0.6901
Epoch 351/2500
22/22 [==============================] - 5s 225ms/step - loss: 0.7114 - categorical_accuracy: 0.6652
Epoch 352/2500
22/22 [==============================] - 5s 218ms/step - loss: 0.6710 - categorical_accuracy: 0.6842
Epoch 353/2500
22/22 [==============================] - 4s 196ms/step - loss: 0.6480 - categorical_accuracy: 0.7164
Epoch 354/2500
22/22 [==============================] - 5s 210ms/step - loss: 0.6517 - categorical_accuracy: 0.7251
Epoch 355/2500
22/22 [==============================] - 4s 189ms/step - loss: 0.7961 - 

22/22 [==============================] - 5s 218ms/step - loss: 0.6997 - categorical_accuracy: 0.7061
Epoch 418/2500
22/22 [==============================] - 5s 223ms/step - loss: 0.6101 - categorical_accuracy: 0.7544
Epoch 419/2500
22/22 [==============================] - 5s 218ms/step - loss: 0.5879 - categorical_accuracy: 0.7368
Epoch 420/2500
22/22 [==============================] - 5s 224ms/step - loss: 0.5805 - categorical_accuracy: 0.7515
Epoch 421/2500
22/22 [==============================] - 5s 241ms/step - loss: 0.5773 - categorical_accuracy: 0.7339
Epoch 422/2500
22/22 [==============================] - 5s 227ms/step - loss: 0.6452 - categorical_accuracy: 0.7544
Epoch 423/2500
22/22 [==============================] - 5s 227ms/step - loss: 0.6387 - categorical_accuracy: 0.7368
Epoch 424/2500
22/22 [==============================] - 5s 218ms/step - loss: 0.5763 - categorical_accuracy: 0.7822
Epoch 425/2500
22/22 [==============================] - 5s 235ms/step - loss: 0.5864 - 

...

22/22 [==============================] - 4s 187ms/step - loss: 0.0030 - categorical_accuracy: 1.0000
Epoch 2014/2500
22/22 [==============================] - 4s 189ms/step - loss: 0.0030 - categorical_accuracy: 1.0000
Epoch 2015/2500
22/22 [==============================] - 4s 189ms/step - loss: 0.0030 - categorical_accuracy: 1.0000
Epoch 2016/2500
22/22 [==============================] - 4s 186ms/step - loss: 0.0028 - categorical_accuracy: 1.0000
Epoch 2017/2500
22/22 [==============================] - 4s 191ms/step - loss: 0.0027 - categorical_accuracy: 1.0000
Epoch 2018/2500
22/22 [==============================] - 4s 197ms/step - loss: 0.0027 - categorical_accuracy: 1.0000
Epoch 2019/2500
22/22 [==============================] - 4s 196ms/step - loss: 0.0026 - categorical_accuracy: 1.0000
Epoch 2020/2500
22/22 [==============================] - 4s 196ms/step - loss: 0.0026 - categorical_accuracy: 1.0000
Epoch 2021/2500
22/22 [==============================] - 4s 194ms/step - loss: 0

22/22 [==============================] - 4s 189ms/step - loss: 0.0039 - categorical_accuracy: 1.0000
Epoch 2084/2500
22/22 [==============================] - 4s 187ms/step - loss: 0.0038 - categorical_accuracy: 1.0000
Epoch 2085/2500
22/22 [==============================] - 4s 189ms/step - loss: 0.0037 - categorical_accuracy: 1.0000
Epoch 2086/2500
22/22 [==============================] - 4s 184ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 2087/2500
22/22 [==============================] - 4s 183ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 2088/2500
22/22 [==============================] - 4s 183ms/step - loss: 0.0034 - categorical_accuracy: 1.0000
Epoch 2089/2500
22/22 [==============================] - 4s 188ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 2090/2500
22/22 [==============================] - 4s 190ms/step - loss: 0.0032 - categorical_accuracy: 1.0000
Epoch 2091/2500
22/22 [==============================] - 4s 184ms/step - loss: 0

22/22 [==============================] - 5s 212ms/step - loss: 0.0010 - categorical_accuracy: 1.0000
Epoch 2223/2500
22/22 [==============================] - 5s 206ms/step - loss: 9.8431e-04 - categorical_accuracy: 1.0000
Epoch 2224/2500
22/22 [==============================] - 5s 207ms/step - loss: 9.7832e-04 - categorical_accuracy: 1.0000
Epoch 2225/2500
22/22 [==============================] - 5s 207ms/step - loss: 9.6638e-04 - categorical_accuracy: 1.0000
Epoch 2226/2500
22/22 [==============================] - 5s 206ms/step - loss: 9.6287e-04 - categorical_accuracy: 1.0000
Epoch 2227/2500
22/22 [==============================] - 5s 207ms/step - loss: 9.4442e-04 - categorical_accuracy: 1.0000
Epoch 2228/2500
22/22 [==============================] - 5s 208ms/step - loss: 9.9385e-04 - categorical_accuracy: 1.0000
Epoch 2229/2500
22/22 [==============================] - 5s 208ms/step - loss: 9.7204e-04 - categorical_accuracy: 1.0000
Epoch 2230/2500
22/22 [=============================

22/22 [==============================] - 4s 188ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 2362/2500
22/22 [==============================] - 4s 188ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 2363/2500
22/22 [==============================] - 4s 187ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 2364/2500
22/22 [==============================] - 4s 186ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 2365/2500
22/22 [==============================] - 4s 188ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 2366/2500
22/22 [==============================] - 4s 188ms/step - loss: 0.0018 - categorical_accuracy: 1.0000
Epoch 2367/2500
22/22 [==============================] - 4s 186ms/step - loss: 0.0018 - categorical_accuracy: 1.0000
Epoch 2368/2500
22/22 [==============================] - 4s 190ms/step - loss: 0.0017 - categorical_accuracy: 1.0000
Epoch 2369/2500
22/22 [==============================] - 4s 195ms/step - loss: 0

22/22 [==============================] - 5s 220ms/step - loss: 5.8871e-04 - categorical_accuracy: 1.0000
Epoch 2498/2500
22/22 [==============================] - 5s 227ms/step - loss: 5.8242e-04 - categorical_accuracy: 1.0000
Epoch 2499/2500
22/22 [==============================] - 4s 205ms/step - loss: 5.7952e-04 - categorical_accuracy: 1.0000
Epoch 2500/2500
22/22 [==============================] - 4s 197ms/step - loss: 5.7502e-04 - categorical_accuracy: 1.0000


model.fit(X_train, y_train, epochs=2500, callbacks=[tb_callback])

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 12)                396       
                                                                 
Total params: 596,972
Trainable params: 596,972
Non-tr

In [34]:
res = model.predict(X_test)

In [35]:
actions[np.argmax(res[0])]

'cerewet'

In [36]:
actions[np.argmax(y_test[0])]

'cerewet'

In [37]:
model.save('action.h5')

In [38]:
model.load_weights('action.h5')

In [39]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [40]:
yhat = model.predict(X_test)

In [41]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [42]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[13,  0],
        [ 0,  2]],

       [[12,  0],
        [ 1,  2]],

       [[ 6,  0],
        [ 0,  9]],

       [[14,  1],
        [ 0,  0]],

       [[14,  0],
        [ 0,  1]]], dtype=int64)

In [43]:
print("Akurasi : ",accuracy_score(ytrue, yhat))

Akurasi :  0.9333333333333333


In [45]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.9

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read(1)

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
lucu
<class 'mediapipe.python.solution_base.SolutionOutputs'>
lucu
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cerewet
<class 'mediapipe.python.soluti

In [ ]:
    cap.release()
    cv2.destroyAllWindows()

In [ ]:


model.predict(np.expand_dims(X_test[0], axis=0))

